<a href="https://colab.research.google.com/github/david-meltzer/LLMs/blob/main/distilgpt2_simplewiki_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/gpt2

/content/drive/MyDrive/gpt2


In [3]:
!ls

distilgpt2_simplewiki  gpt2_finetune.ipynb  wiki_simple


In [4]:
!pip install transformers[torch]
!pip install datasets
!pip install apache_beam
!pip install mwparserfromhell

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.8 MB/s eta 0

In [5]:
from pprint import pprint as pp

In [6]:
from datasets import load_dataset
wiki = load_dataset("wikipedia", "20220301.simple", split = 'train')
#wiki_small = wiki.select(range(5000))
#wiki_small.save_to_disk('wiki_small')

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235M [00:00<?, ?B/s]

In [ ]:
wiki.save_to_disk('wiki_simple')

Saving the dataset (0/1 shards):   0%|          | 0/205328 [00:00<?, ? examples/s]

In [7]:
wiki_split = wiki.train_test_split(test_size = 5000)
wiki_split

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 200328
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 5000
    })
})

In [9]:
wiki_split['train'][40]

{'id': '754882',
 'url': 'https://simple.wikipedia.org/wiki/Namosi%20Province',
 'title': 'Namosi Province',
 'text': 'Namosi is a province in Fiji. It is on Viti Levu, the largest island. It is part of the Central Division.\n\nThe province covers 570 square kilometers. About 7,800 people lived there in 2017. Namosi Province includes Namosi District, as well as the districts of Veivatuloa and Wainikoroiluva.\n\nProvinces of Fiji'}

In [17]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
pp(tokenizer)
pp(tokenizer.model_max_length)
pp(tokenizer.is_fast)

GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)
1024
True


In [18]:
context_len = 512

def tokenize(element):
    # outputs - BatchEncoding with keys: input_ids, attention_mask, length
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=context_len,
        return_overflowing_tokens=True,
        return_length=True,
    )

    return outputs


wiki_tokenized = wiki_split.map(
    tokenize, batched = True, num_proc = 4, remove_columns=wiki_split["train"].column_names
)

wiki_tokenized

Map (num_proc=4):   0%|          | 0/200328 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping'],
        num_rows: 242938
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping'],
        num_rows: 6102
    })
})

In [24]:
pp(wiki_tokenized['train'][11], compact = True, width = 150)

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [24857, 8107, 3914, 33577, 13876, 694, 64, 357, 6286, 3267, 1679, 11, 13521, 8, 318, 257, 5398, 4708, 34845, 11, 1188, 316, 290, 1966,
               2746, 13, 1375, 3058, 10419, 829, 329, 15247, 319, 663, 2439, 441, 8048, 4508, 739, 262, 5858, 1438, 11, 27994, 20442, 274, 13, 1375,
               925, 607, 15247, 8886, 355, 636, 286, 262, 4793, 4777, 64, 11140, 11, 772, 996, 673, 5201, 5544, 15247

#Model

In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [26]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

#Train

In [27]:
from transformers import TrainingArguments, Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir="distilgpt2_simplewiki",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,
#   per_device_train_batch_size=8,  # default batch size per device during training, batch size = 16 doesn't fit on T4 GPU
#   per_device_eval_batch_size=8,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=wiki_tokenized["train"],
    eval_dataset=wiki_tokenized["test"],
    data_collator=data_collator,
)

trainer.train()